In [1]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Masking
import pandas as pd
import numpy as np
from keras.layers import Dense, Activation, Conv1D, MaxPool1D, Embedding, LSTM, SimpleRNN, Flatten, Input, Activation
from keras.layers import TimeDistributed, AveragePooling1D, Dropout, Reshape
from keras.models import Sequential


Using TensorFlow backend.


# Read dataset
the dataset is from kaggle. It is a table of information. 

In [88]:
data = pd.read_table('dataset/training.txt', names=["y", "x"])
y, x = data["y"], data["x"]
tmpx, tmpy = x, y
# x, y

# Let's prepare dataset

In [3]:
from keras.preprocessing.text import text_to_word_sequence, Tokenizer

from keras.utils import to_categorical
import math

# Now I have to create vectors for all sequences


In [57]:


tok = Tokenizer()
tok.fit_on_texts(x)

# tt.texts_to_sequences(t)
seq  = tok.texts_to_sequences(x)
# all_x = pad_sequences(seq,maxlen = mx)
word_index = tok.word_index
# splitting data in two category positive or negative
v = len(tok.word_index)
# all_x, all_y

In [109]:
word_index['the']

2

In [89]:
# np.split?
x, y  =tmpx, tmpy
clusters = []
max_len = 80
for i in seq:
    i = np.array(i)
    l = math.ceil(len(i) / max_len)
    padded = pad_sequences([i], l * max_len, padding="post")[0]
    padded = padded.reshape((-1, max_len))
    clusters.append(padded)
clusters = np.array(clusters)
clusters
all_y = to_categorical(y, num_classes=2)

### shuffling
The code below is for shuffling data
the traning data is having all negative sentences aligned to the top and all the 
 positive sentences in bottom of dataset. I zip all sentences and make numpy array and shuffle it with `numpy.random.shuffle()` method

In [42]:
clusters[100], all_y[100]

(array([[  1,  48,   7,   6,   8, 651,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0]], dtype=int32), array([ 0.,  1.]))

In [58]:
all_x

array([[  0,   0,   0, ...,  12,  40,  15],
       [  0,   0,   0, ..., 956, 329,  56],
       [  0,   0,   0, ...,   8,  13, 311],
       ..., 
       [  0,   0,   0, ...,  20,   9,  10],
       [  0,   0,   0, ...,  13,  70,  20],
       [  0,   0,   0, ...,  13,  78,  20]], dtype=int32)


### Why transpose here ?
Since I shuffled data. the data is noww having shape (6000,2). To separate all traning data and expected outputs I perform transpose on data, which results in `numpy.array` of shape (2, 6000). that's pretty good for me.

In [43]:


r_x, r_y, r_x.shape
r_x.shape

NameError: name 'r_x' is not defined

### generating test, train dataset
since all positive data and negative are together but not shuffled, we gotta shuffle it here. Then we split dataset into test and train parts.

# Import required Layers for writing Model

In [44]:
from keras.layers import Conv2D, MaxPool2D

In [97]:
model = Sequential()  
model.add(LSTM( 256, return_sequences=True, input_shape=(None, max_len)))  
model.add(LSTM( 256,  input_shape=(None, max_len)))  
model.add(Dense(256, activation = "tanh"))  
model.add(Dense(2))

model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, None, 256)         345088    
_________________________________________________________________
lstm_25 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_25 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_26 (Dense)             (None, 2)                 514       
Total params: 936,706
Trainable params: 936,706
Non-trainable params: 0
_________________________________________________________________


In [90]:
def prepare(clusters, all_y):
    all_data = np.array(list(zip(clusters, all_y)))
    np.random.shuffle(all_data)
    r_x, r_y = all_data.T


    x_train = r_x[:6000] # first 6000 examples for training
    x_test  = r_x[6000:] # len - 6000 for evaluation /testing
    y_train = r_y[:6000]
    y_test  = r_y[6000:]

    px, py = [], []
    for x, y in zip(x_train, y_train):
        x = np.array([[ [*i] for i in x]]) / (v+1)
        y = np.array([[*y]])
        px.append(x)
        py.append(y)
    tx, ty = [], []
    for x, y in zip(x_test, y_test):
        x = np.array([[ [*i] for i in x]]) / (v+1)
        y = np.array([[*y]])
        y.reshape((-1,2))
        tx.append(x)
        ty.append(y)
    return px, py, tx, ty

In [91]:
import matplotlib.pyplot as plt
plt.ion()

# plt.axis([0, 10, 0, 1])

In [99]:
epoch_count = 100
for i in range(epoch_count):
    px, py, tx, ty = prepare(clusters, all_y)
    
    for i in range(10):
        lc, ac = 0, 0
        for x, y in zip(tx, ty):
            loss, accuracy = model.evaluate(x, y, verbose=0)
            if loss > accuracy:
                lc += 1
            else:
                ac += 1
            print("{}:{} {}{} {}{}{}".format('Iteration', (lc + ac), 'loss:',loss,'accuracy:',accuracy, ' '*50), end='\r')
        print('Testing epoch :', i,'loss :',lc / (lc + ac), 'accuracy:', ac / (lc + ac),  ' '* 10)
        lc, ac = 0, 0
        for x, y in zip(px[:1000], py):
            loss, accuracy = model.evaluate(x, y, verbose=0)
            model.fit(x,y, batch_size= 1, verbose = 0, epochs= 1)
            if loss > accuracy:
                lc += 1
            else:
                ac += 1
            print("{}:{} {}{} {}{}{}".format('Iteration', (lc + ac), 'loss:',loss,'accuracy:',accuracy, ' '*50), end='\r')
        print('Training epoch :', i,'loss :',lc / (lc + ac), 'accuracy:', ac / (lc + ac), ' '* 10)

    
    

Testing epoch : 0 loss : 0.19934640522875818 accuracy: 0.8006535947712419                               
Training epoch : 0 loss : 0.199 accuracy: 0.801                                                         
Testing epoch : 1 loss : 0.20806100217864923 accuracy: 0.7919389978213508                               
Training epoch : 1 loss : 0.193 accuracy: 0.807                                                         
Testing epoch : 2 loss : 0.20043572984749455 accuracy: 0.7995642701525054                               
Training epoch : 2 loss : 0.187 accuracy: 0.813                                                         
Testing epoch : 3 loss : 0.19716775599128541 accuracy: 0.8028322440087146                               
Training epoch : 3 loss : 0.178 accuracy: 0.822                                                         
Testing epoch : 4 loss : 0.19934640522875818 accuracy: 0.8006535947712419                               
Training epoch : 4 loss : 0.171 accuracy: 0.829        

Testing epoch : 7 loss : 0.10130718954248366 accuracy: 0.8986928104575164                               
Training epoch : 7 loss : 0.106 accuracy: 0.894                                                         
Testing epoch : 8 loss : 0.11002178649237472 accuracy: 0.8899782135076253                               
Training epoch : 8 loss : 0.105 accuracy: 0.895                                                          
Testing epoch : 9 loss : 0.10457516339869281 accuracy: 0.8954248366013072                               
Training epoch : 9 loss : 0.099 accuracy: 0.901                                                         
Testing epoch : 0 loss : 0.1328976034858388 accuracy: 0.8671023965141612                                


KeyboardInterrupt: 

In [106]:
predict = pd.read_table("./dataset/training.txt", names = ['x', 'y'])
# predict = predict['x']
predict

,x,y
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...
5,1,that's not even an exaggeration ) and at midni...
6,1,"I loved the Da Vinci Code, but now I want some..."
7,1,"i thought da vinci code was great, same with k..."
8,1,The Da Vinci Code is actually a good movie...
9,1,I thought the Da Vinci Code was a pretty good ...


In [102]:
tok1 = Tokenizer()
tok1.fit_on_texts(predict)
# tt.texts_to_sequences(t)
seq1  = tok1.texts_to_sequences(predict)
# all_x = pad_sequences(seq,maxlen = mx)
word_index1 = tok1.word_index
# splitting data in two category positive or negative
v1 = len(tok1.word_index)

clusters1 = []
for i in seq1:
    i = np.array(i)
    l = math.ceil(len(i) / max_len)
    padded = pad_sequences([i], l * max_len, padding="post")[0]
    padded = padded.reshape((-1, max_len))
    clusters1.append(padded)
clusters1 = np.array(clusters1)
# all_y1 = to_categorical(y, num_classes=2)

prx = []
for x in clusters1:
    x = np.array([[ [*i] for i in x]]) / (v1+1)
    prx.append(x)

In [103]:
for i, e in enumerate(prx):
    p, n = model.predict(e)[0]
    if p < n:
        print("positive : ", end=" ")
    else:
        print("Negative : ", end=" ")
    print([p, n] ,predict[i])
    

Negative :  [0.92453599, 0.17342976]  I don't care what anyone says, I like Hillary Clinton.
have an awesome time at purdue!..
Yep, I'm still in London, which is pretty awesome: P Remind me to post the million and one pictures that I took when I get back to Markham!...
Have to say, I hate Paris Hilton's behavior but I do think she's kinda cute..
i will love the lakers.
I'm so glad I love Paris Hilton, too, or this would be excruciating.
considering most Geico commericals are stupid...
i liked MIT though, esp their little info book(
Before I left Missouri, I thought London was going to be so good and cool and fun and a really great experience and I was really excited.
I still like Tom Cruise.
well, i had a piece of crap toyota celica but it died in portland and i got a ford ranger..
i love angelina jolie.
I still like Tom Cruise.
UCLA is beautiful.
I think Angelina Jolie is so much more beautiful than Jennifer Anniston, who, by the way, is majorly OVERRATED.
Angelina Jolie is beautiful.

positive :  [0.074005246, 0.86637306] Stupid UCLA....
positive :  [0.053570241, 0.95642328] I miss crossing London's bridges ( and am suffering severe case of London homesickness at the moment...! ).
positive :  [-0.019610554, 1.0406841] i need clothes before i look stupid @ lakers....
positive :  [0.016798943, 0.97773522] i love seattle so much.
positive :  [0.046622396, 0.94253379] I'm not crazy about HK either, but Shanghai is sounding awesome.
Negative :  [1.0339745, -0.029035769] I had to tape because my mom had to watch her stupid Boston Legal and its a new TV / VCR so I was taping the wrong channel.
positive :  [0.36714783, 0.64443332] GEICO is a GREAT ride.
positive :  [0.017821699, 0.97717702] i do i love angelina jolie!..
positive :  [0.048592865, 0.91684234] I love Angelina Jolie.
positive :  [0.27865997, 0.70468396] San Francisco Coffee rocks, yo!
positive :  [0.089181662, 0.82154286] Shanghai is boring..
positive :  [-0.13488635, 0.99655759] the fact that i love paris hilt

positive :  [0.14344093, 0.8467527] Paris Hilton – Sexy, relevant to a different kind of Bar...
positive :  [-0.064001113, 1.0799472] he loves NYC too much in my opinion ( because i'll always love my seattle too much )...
positive :  [0.016798943, 0.97773522] i love seattle so much.
positive :  [0.45946962, 0.52370518] I'd like to talk today about how much I hate Paris Hilton.....
Negative :  [0.59138358, 0.37505895] we have a boring as shit blue 2005 toyota carolla.
positive :  [0.38558871, 0.58203763] I love Harvard Square in the fall.
positive :  [0.0091428459, 0.98042756] And Tom Cruise is beautiful....
Negative :  [0.75459361, 0.27887335] I liked Tom Cruise until he dumped Nicole Kidman.
positive :  [0.15098938, 0.8291325] I absolutely love my MacBook Pro.
Negative :  [0.70865685, 0.24917665] usually, i can't at my house but i'm at the library but i moved into my aunts house in san carlos to go to school here because schools in san francisco sucks BIG TIME!
positive :  [0.09235122

positive :  [0.06715551, 0.89722943] I hate the Lakers...
Negative :  [0.56139994, 0.4998475] Before I left Missouri, I thought London was going to be so good and cool and fun and a really great experience and I was really excited.
positive :  [0.27990627, 0.64485347] I used to like Tom Cruise but the Matt Lauer interview really turned me off.
Negative :  [0.98061156, 0.05034738] OH AND LAKERS SUCK!..
Negative :  [0.96393108, 0.029064946] By the time I left the Hospital, I had no time to drive to Boston-which sucked because 1)
positive :  [0.27767289, 0.75427616] Locating the most select aaa travel norfolk virginia sites is frequently difficult.
Negative :  [0.85372049, 0.1522541] For those who say he doesn't deserve it cuz the Lakers stink and are barely over.
positive :  [-0.068730444, 1.0709435] as title, tho i hate london, i did love alittle bit about london..
Negative :  [0.80832994, 0.2869668] There is a Taste of China right by GEICO that has great food.
Negative :  [0.85191613, 

positive :  [0.47126949, 0.66790599] then for ten minutes i have to listen to brandon talk about how ucla sucks.
Negative :  [0.60768449, 0.31531981] I'm a big fan of Lakers, so I kind of have all the schedule for this season which it's end in april...
positive :  [0.29601038, 0.69534326] Aber das mit Reno und Rude war auf der BK auch klasse ^.
positive :  [0.13832176, 0.85009855] I love San Francisco, but getting art in New York comes first...
positive :  [0.34469366, 0.67287678] & i adore my little honda < 3.
Negative :  [0.67111796, 0.41920596] Prolly going to Cambridge on Tuesday, I need to see MIT to survive till the rest of the term.
Negative :  [0.54433823, 0.42842522] Angelina Jolie is beautiful.
positive :  [0.49848023, 0.56360191] While we're on the subject, I'd like to denounce Harvard as anymore worthwhile than any other higher education institution.
positive :  [-0.0172185, 1.0418131] My Purdue Cal friends are awesome!..
Negative :  [0.61534798, 0.38512218] i look across m

...

Negative :  [0.73632979, 0.28729418] Well apparently all people driving toyota 4runners are appalingly ugly, or so one would think based on him not even glancing in my direction!
positive :  [0.074005246, 0.86637306] stupid UCLA..
positive :  [0.45946962, 0.52370518] I'd like to talk today about how much I hate Paris Hilton.....
Negative :  [0.74743772, 0.26480505] besides, UCLA was a stupid dream...
positive :  [0.34139392, 0.61946219] It was really ironic that he spent the first part of class talking about his own professor at Harvard who was a pompous arrogant ass.
positive :  [0.092122912, 0.88383472] I can see why he might have had problems with Karen but Toyota Alexa is useless and Jo is MENTAL!!!
positive :  [0.0006069243, 0.97147501] I LOVE Seattle!!
positive :  [0.48613092, 0.69782388]  AAA is useless Mike  --- Joe and Anthony..
positive :  [0.18059403, 0.74727792] Tom Cruise sucks.
positive :  [0.032850564, 0.93995303] I also like Boston.
positive :  [0.42069641, 0.55902714] 

positive :  [0.034194171, 1.007282] I love the Toyota Prius.
positive :  [-0.13438848, 1.1563258] I loved the Lakers episode where he told that little blonde he wants to see a woman rule the world in his lifetime because women know how to look after the pack.
positive :  [0.38427329, 0.5609231] Then we had stupid trivia about San Francisco and sci fi tv so we left...
Negative :  [0.71105897, 0.3087762] Harvard sucked a lot.
positive :  [0.32176021, 0.55790383] aaa prolly spelled wrong but I don't care....
Negative :  [0.59493536, 0.36844134] i think if i hate boston, ill just disappear to california for the summers...
positive :  [0.46876344, 0.50159657] yeah, I'm already an hour away from T. O. so it's not close for me to go to kitchener or London which sucks...
positive :  [0.0006069243, 0.97147501] I LOVE Seattle!!
Negative :  [0.80896056, 0.20217115] I needed San Francisco but not until a month ago.
positive :  [0.027672201, 0.92051655] But I miss Boston.
Negative :  [0.60768449, 0

positive :  [0.043750465, 0.94808203] I still like Tom Cruise.
positive :  [0.24639431, 0.70978761] boston college is good too < 33.
Negative :  [0.70865685, 0.24917665] usually, i can't at my house but i'm at the library but i moved into my aunts house in san carlos to go to school here because schools in san francisco sucks BIG TIME!
positive :  [0.43532726, 0.49334985] I should remind that self that my boring green'96 Toyota Camry has 230k on the odometer, hasn't been to get a tune-up in over a year -- poor thing;
positive :  [0.31753689, 0.66636574] I definitely want a Volkswagen van...
positive :  [0.12880558, 0.8024978] Boston SUCKS.
positive :  [0.092122912, 0.88383472] I can see why he might have had problems with Karen but Toyota Alexa is useless and Jo is MENTAL!!!
positive :  [0.032425255, 0.93626845] lakers suck..-_-but i still like them haha.
positive :  [0.091899931, 0.90259683] but it's fine.. ( though i miss my sweet sick honda.. )..
positive :  [0.25822055, 0.9682669] 

positive :  [0.036616385, 0.9446798] I need some of that geico balboa stuff..
positive :  [0.016798943, 0.97773522] i love seattle so much.
positive :  [0.071191698, 0.9022699] I miss Boston so much it hurts sometimes, but I know that I'm really just missing this time of my life and the people from it...
positive :  [0.19457589, 0.80087805] I like Purdue University's computer lab!..
positive :  [0.22583678, 0.70734388] not only do i hate paris hilton for....
positive :  [0.0058860779, 0.96929806] purdue and ohio were superb!
positive :  [0.040892035, 0.93871993] i love shanghai, want to go back..
positive :  [-0.13216677, 1.142482] I worship Nicole Richie or Paris Hilton..
positive :  [0.0006069243, 0.97147501] i love seattle..
positive :  [0.088020355, 0.83203161] Have I told you I hate Tom Cruise?..
Negative :  [0.60453784, 0.39435813] When Emmy was in town, I told her ( in a moment of pure physical pain as we hiked up to the Lombard / Leavenworth intersection ) that San Francisco is

KeyboardInterrupt: 

# model(1) 90% on test data
```
model = Sequential()  
model.add(LSTM( 128, return_sequences=True, input_shape=(None, max_len)))  
model.add(LSTM( 128,  input_shape=(None, max_len)))  
model.add(Dense(128, activation = "tanh"))  
model.add(Dense(2))

model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()
```

In [108]:
model.save_weights('model2.hdf')